In [1]:
from TwitterSearch import *
import string
from string import digits
import Stemmer
import re
import pandas as pd
import numpy as np

In [111]:
import nltk

In [2]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.cross_validation import train_test_split


In [51]:
import word2vec

In [ ]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


Exception in Tkinter callback
Traceback (most recent call last):
  File "/Applications/anaconda/lib/python3.5/tkinter/__init__.py", line 1550, in __call__
    return self.func(*args)
  File "/Applications/anaconda/lib/python3.5/tkinter/__init__.py", line 596, in callit
    func(*args)
  File "/Applications/anaconda/lib/python3.5/site-packages/nltk/downloader.py", line 1914, in _monitor_message_queue
    self._select(msg.package.id)
AttributeError: 'str' object has no attribute 'id'


In [ ]:
from nltk.corpus import stopwords # Import the stop word list
print (stopwords.words("russian") )

In [3]:
tweets=[]
try:
    tso = TwitterSearchOrder() # create a TwitterSearchOrder object
    tso.set_keywords(['Форсаж8']) # define all words we would like to have a look for
    tso.set_language('ru') # russian tweets only
    tso.set_include_entities(False) # and don't give us all those entity information

    ts = TwitterSearch(
        consumer_key = 'XfyrnpYx2jgkvqVMO28Dl4vro',
        consumer_secret = 'DNU3vVeboWDmRIC7PBi08j2hpK05mhiw5cYHRIJZqOUIKQ0hob',
        access_token = '410867738-Yeyao2EBTjBVxruKM9Zw7MrvKmmkRxmbeHm1GGlx',
        access_token_secret = 'sxxY0nVKVt9o0JbZGojJAEG6yGHHIKykOaDImOtGPAWPq'
     )

     
    for tweet in ts.search_tweets_iterable(tso):
        tweets.append('%s'%(tweet['text']))

except TwitterSearchException as e: # take care of all those ugly errors if there are some
    print(e)

In [4]:
tweets=list(set(tweets))

In [41]:
tweets

['Какой же пиздатый фильм, я в шоке!\nСпасибо огромнейшее за этот шедевр!\n#форсаж8 #Furious8',
 'Сцена в тюрьме - огонь! 🔥🔥🔥\n#форсаж8',
 'Х/ф #форсаж8 очень красивый и невероятно тупой. Как твоя бывшая',
 'Ребят ,,форсаж 8" это идеальный фильм оооооооооо😍😍😍😙😘😚😙😘😍 \n#форсаж8',
 'А это мы сегодня с Булкой смотрели #форсаж8😍 Фильмец,кст,зашел👍Очень клевый, не хуже остальных\nОсталось приятное вп… https://t.co/Mz45EIjZdT',
 'Никогда не посмотрю "форсаж8"',
 'Итак, предлагаю не продлевать франшизу на 9-ю серию. #форсаж8',
 '#форсаж8 полная чушь, не о чем! Но когда будет#форсаж9 , я скорее всего опять пойду :)',
 'RT @Medialogia: Какие фильмы больше всего обсуждали в соцмедиа в апреле? #Форсаж8 https://t.co/dTni2CyxJp',
 'форсаж8 чисто поржать https://t.co/xZEKLbzbn6',
 'Чтоооо? Такой крышеснос от одного фильма. Еаахх. #форсаж8. Режиссеры конечно постарались...',
 'Еееееее,дождалась наконец-то!!! Легендарный Форсаж 8. Погоняем)) #форсаж8 https://t.co/GSYKwxyFOc',
 'Форсаж8.в след части он 

In [83]:
def clear_tweets(tweets):
    tags=['_','__','\n']
    remove_digits = str.maketrans('', '', digits)
    remove_latins=str.maketrans('','','abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ')
    exclude = set(string.punctuation)
    cyrillic=set('абвгдеёжзийклмнопрстуфхцчшщъыьэюя')
    #clear tweets from digits, punctuation, latin words. Just splitted words
    clr_tweets=[]
    for s in tweets:
        s=s.translate(remove_digits)
        s=s.translate(remove_latins)
        s=s[:s.find('https://')]
        s=s.lower()
        #s = ''.join(ch for ch in s if ch not in exclude)
        
        s=re.sub('\W',' ', s)
        a=re.sub('_',' ',s)
        #s=s.replace('\n',' ')
        #for i in s:
            #if i not in cyrillic:
               # s=s.replace(i,' ')
        clr_tweets.append(a)
        
    return (clr_tweets)


In [84]:
new_tweets=clear_tweets(tweets)

In [85]:
positive=pd.read_csv('positive.csv', encoding='utf-8',sep=';', header=None,usecols=[3,4])
negative=pd.read_csv('negative.csv', encoding='utf-8',sep=';', header=None,usecols=[3,4])
pos_tweets=clear_tweets(list(positive.iloc[:,0]))
neg_tweets=clear_tweets(list(negative.iloc[:,0]))

In [86]:
all_tweets=new_tweets+pos_tweets+neg_tweets

In [87]:
all_tweets

['какой же пиздатый фильм  я в шоке  спасибо огромнейшее за этот шедевр   форсаж ',
 'сцена в тюрьме   огонь       форса',
 'х ф  форсаж очень красивый и невероятно тупой  как твоя бывша',
 'ребят   форсаж   это идеальный фильм оооооооооо            форса',
 'а это мы сегодня с булкой смотрели  форсаж  фильмец кст зашел очень клевый  не хуже остальных осталось приятное вп      ',
 'никогда не посмотрю  форсаж',
 'итак  предлагаю не продлевать франшизу на  ю серию   форса',
 ' форсаж полная чушь  не о чем  но когда будет форсаж   я скорее всего опять пойду  ',
 '    какие фильмы больше всего обсуждали в соцмедиа в апреле   форсаж     ',
 'форсаж чисто поржать     ',
 'чтоооо  такой крышеснос от одного фильма  еаахх   форсаж  режиссеры конечно постарались  ',
 'еееееее дождалась наконец то    легендарный форсаж   погоняем    форсаж     ',
 'форсаж в след части он будет всех делать на велике  или на колесе',
 'я одна еще не ходила на форсаж',
 'все так обсуждают  форсаж  а я не смотрела  

In [88]:
f = open('fastfurious.txt', 'w')
for index in all_tweets:
    f.write(index + '\n')
f.close()

In [89]:
word2vec.word2phrase('fastfurious.txt', 'fast-phrases2', verbose=True)

b'Starting training using file fastfurious.txt\n'b'Words processed: 100K     Vocab size: 98K  \rWords processed: 200K     Vocab size: 177K  \rWords processed: 300K     Vocab size: 249K  \rWords processed: 400K     Vocab size: 317K  \rWords processed: 500K     Vocab size: 380K  \rWords processed: 600K     Vocab size: 442K  \rWords processed: 700K     Vocab size: 503K  \rWords processed: 800K     Vocab size: 561K  \rWords processed: 900K     Vocab size: 619K  \rWords processed: 1000K     Vocab size: 674K  \rWords processed: 1100K     Vocab size: 728K  \rWords processed: 1200K     Vocab size: 782K  \rWords processed: 1300K     Vocab size: 835K  \rWords processed: 1400K     Vocab size: 885K  \rWords processed: 1500K     Vocab size: 932K  \rWords processed: 1600K     Vocab size: 977K  \rWords processed: 1700K     Vocab size: 1021K  \rWords processed: 1800K     Vocab size: 1066K  \rWords processed: 1900K     Vocab size: 1110K  \rWords processed: 2000K     Vocab size: 1152K  \rWords processed

In [90]:
word2vec.word2vec('fast-phrases2', 'fastfurious2.bin', size=100, verbose=True)

b'Starting training using file fast-phrases2\n'b'100K\r200K\r300K\r400K\r500K\r600K\r700K\r800K\r900K\r1000K\r1100K\r1200K\r1300K\r1400K\r1500K\r1600K\r1700K\r1800K\r1900K\r2000K\r2100K\r2200K\r2300K\r2400K\r2500K\r2600K\rVocab size: 35143\n'b'Words in train file: 2420914\n'b'\rAlpha: 0.025000  Progress: 0.08%  Words/thread/sec: 30.22k  \rAlpha: 0.024979  Progress: 0.17%  Words/thread/sec: 45.18k  \rAlpha: 0.024959  Progress: 0.25%  Words/thread/sec: 64.06k  \rAlpha: 0.024938  Progress: 0.33%  Words/thread/sec: 84.53k  \rAlpha: 0.024917  Progress: 0.41%  Words/thread/sec: 100.28k  \rAlpha: 0.024897  Progress: 0.50%  Words/thread/sec: 109.57k  \rAlpha: 0.024876  Progress: 0.58%  Words/thread/sec: 116.61k  \rAlpha: 0.024855  Progress: 0.66%  Words/thread/sec: 132.89k  \rAlpha: 0.024835  Progress: 0.74%  Words/thread/sec: 146.43k  \rAlpha: 0.024814  Progress: 0.83%  Words/thread/sec: 160.20k  \rAlpha: 0.024793  Progress: 0.91%  Words/thread/sec: 163.37k  \rAlpha: 0.024773  Progress: 0.99%

In [91]:
model = word2vec.load('fastfurious2.bin')

In [80]:
model['части']

array([ 0.09558336, -0.06971632,  0.06923366,  0.0217668 ,  0.16871561,
       -0.0885165 , -0.01203335, -0.10472785, -0.07014643,  0.0930474 ,
       -0.04766385,  0.24757943,  0.06841858, -0.02525415, -0.0853277 ,
        0.2083526 ,  0.19005786,  0.05465203,  0.2364147 ,  0.02513018,
       -0.0083151 ,  0.04809021,  0.1413004 , -0.06241001,  0.00782415,
       -0.11302087, -0.01221053, -0.05818186,  0.12069415,  0.10844323,
        0.0431887 , -0.02377714, -0.14880849, -0.01443244,  0.03999298,
       -0.10411953,  0.12839064, -0.08515661,  0.04511029, -0.0642531 ,
       -0.10969313,  0.12351376, -0.18212411,  0.10957831,  0.07445213,
        0.0364385 ,  0.13297153,  0.12744544, -0.04318491, -0.12587617,
       -0.00389251, -0.08165751,  0.08366656,  0.03416076, -0.04029524,
       -0.07228224,  0.04746066,  0.14396454,  0.07925028, -0.26460582,
        0.07654162, -0.09558665,  0.0776912 ,  0.06503893, -0.00362969,
        0.08398183,  0.04043456, -0.00115712,  0.01616818, -0.01

In [93]:
indexes, metrics = model.cosine('машина')
model.generate_response(indexes, metrics).tolist()

[('завелась', 0.8887952669524666),
 ('живая', 0.8688992616467711),
 ('авария', 0.8599750460206248),
 ('комната', 0.8594815700595385),
 ('небольшая', 0.8587311800268416),
 ('выглянула', 0.8575072891385835),
 ('оставила', 0.8553647218436162),
 ('огромная', 0.8530542493316228),
 ('нахлынула', 0.8530266421427114),
 ('спина_затекла', 0.8517824812935588)]

In [ ]:
poslist=[s.split() for s in pos_tweets]
neglist=[s.split() for s in neg_tweets]


In [96]:
len(model.vocab)/5

7028.6

In [97]:
word2vec.word2clusters('fastfurious.txt', 'fastfusious-clusters.txt', 7000, verbose=True)

b'Starting training using file fastfurious.txt\n'b'100K\r200K\r300K\r400K\r500K\r600K\r700K\r800K\r900K\r1000K\r1100K\r1200K\r1300K\r1400K\r1500K\r1600K\r1700K\r1800K\r1900K\r2000K\r2100K\r2200K\r2300K\r2400K\r2500K\r2600K\r2700K\rVocab size: 32192\n'b'Words in train file: 2505927\n'b'\rAlpha: 0.025000  Progress: 0.08%  Words/thread/sec: 9.89k  \rAlpha: 0.024980  Progress: 0.16%  Words/thread/sec: 19.69k  \rAlpha: 0.024960  Progress: 0.24%  Words/thread/sec: 24.95k  \rAlpha: 0.024940  Progress: 0.32%  Words/thread/sec: 31.41k  \rAlpha: 0.024920  Progress: 0.40%  Words/thread/sec: 38.62k  \rAlpha: 0.024900  Progress: 0.48%  Words/thread/sec: 45.98k  \rAlpha: 0.024880  Progress: 0.56%  Words/thread/sec: 52.59k  \rAlpha: 0.024860  Progress: 0.64%  Words/thread/sec: 59.65k  \rAlpha: 0.024840  Progress: 0.72%  Words/thread/sec: 66.87k  \rAlpha: 0.024820  Progress: 0.80%  Words/thread/sec: 73.38k  \rAlpha: 0.024800  Progress: 0.88%  Words/thread/sec: 80.13k  \rAlpha: 0.024780  Progress: 0.96

In [98]:
clusters = word2vec.load_clusters('fastfusious-clusters.txt')

In [110]:
clusters

KeyError: 'Word not in vocabulary'

In [108]:
for i in all_tweets[0].split():
    clusters[i]

KeyError: 'Word not in vocabulary'